## Raziskovanje gručenja K-Means z uporabo R in načel urejenih podatkov.

### [**Predhodni kviz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

V tej lekciji se boste naučili, kako ustvariti gruče z uporabo paketa Tidymodels in drugih paketov v ekosistemu R (imenovali jih bomo prijatelji 🧑‍🤝‍🧑) ter nigerijskega glasbenega nabora podatkov, ki ste ga uvozili prej. Pokrili bomo osnove K-Means za gručenje. Upoštevajte, da, kot ste se naučili v prejšnji lekciji, obstaja veliko načinov za delo z gručenjem, metoda, ki jo uporabite, pa je odvisna od vaših podatkov. Poskusili bomo K-Means, saj je to najpogostejša tehnika gručenja. Začnimo!

Pojmi, o katerih se boste učili:

-   Silhuetno ocenjevanje

-   Metoda komolca

-   Inercija

-   Varianca

### **Uvod**

[K-Means gručenje](https://wikipedia.org/wiki/K-means_clustering) je metoda, ki izhaja iz področja obdelave signalov. Uporablja se za razdelitev in razvrščanje skupin podatkov v `k gruče` na podlagi podobnosti njihovih značilnosti.

Gruče je mogoče vizualizirati kot [Voronoijeve diagrame](https://wikipedia.org/wiki/Voronoi_diagram), ki vključujejo točko (ali 'seme') in njeno ustrezno regijo.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika avtorice Jen Looper</figcaption>


Postopek K-Means gručenja vključuje naslednje korake:

1.  Podatkovni znanstvenik najprej določi želeno število gruč, ki jih želi ustvariti.

2.  Nato algoritem naključno izbere K opazovanj iz nabora podatkov, ki služijo kot začetna središča gruč (tj. centroidi).

3.  Nato se vsako preostalo opazovanje dodeli najbližjemu centroidu.

4.  Nato se izračunajo nove povprečne vrednosti vsake grupe, centroid pa se premakne na povprečje.

5.  Ko so središča ponovno izračunana, se vsako opazovanje ponovno preveri, ali bi lahko bilo bližje drugi gruč. Vsa opazovanja se ponovno prerazporedijo z uporabo posodobljenih povprečnih vrednosti gruč. Koraki dodeljevanja gruč in posodabljanja centroidov se ponavljajo, dokler se dodelitve gruč ne prenehajo spreminjati (tj. ko je dosežena konvergenca). Algoritem se običajno ustavi, ko vsaka nova iteracija povzroči zanemarljivo premikanje centroidov in gruče postanejo statične.

<div>

> Upoštevajte, da zaradi naključnosti začetnih k opazovanj, ki se uporabljajo kot začetni centroidi, lahko dobimo nekoliko različne rezultate vsakič, ko uporabimo postopek. Zaradi tega večina algoritmov uporablja več *naključnih začetkov* in izbere iteracijo z najnižjim WCSS. Zato je močno priporočljivo, da K-Means vedno izvajate z več vrednostmi *nstart*, da se izognete *nezaželenemu lokalnemu optimumu.*

</div>

Ta kratka animacija z uporabo [ilustracij](https://github.com/allisonhorst/stats-illustrations) Allison Horst pojasnjuje postopek gručenja:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Ilustracija avtorice @allison_horst</figcaption>



Osnovno vprašanje, ki se pojavi pri gručenju, je naslednje: kako veste, na koliko gruč razdeliti svoje podatke? Ena od pomanjkljivosti uporabe K-Means je dejstvo, da morate določiti `k`, torej število `centroidov`. Na srečo metoda `komolca` pomaga oceniti dobro začetno vrednost za `k`. Kmalu jo boste preizkusili.

### 

**Predpogoj**

Nadaljevali bomo tam, kjer smo končali v [prejšnji lekciji](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), kjer smo analizirali nabor podatkov, ustvarili veliko vizualizacij in filtrirali nabor podatkov na zanimiva opazovanja. Prepričajte se, da si jo ogledate!

Za dokončanje tega modula bomo potrebovali nekaj paketov. Namestite jih lahko z ukazom: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativno spodnji skript preveri, ali imate potrebne pakete za dokončanje tega modula, in jih namesti, če manjkajo.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Začnimo!

## 1. Ples s podatki: Omejimo se na 3 najbolj priljubljene glasbene zvrsti

To je povzetek tega, kar smo naredili v prejšnji lekciji. Razčlenimo in analizirajmo nekaj podatkov!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 To je šlo odlično!

## 2. Več raziskovanja podatkov.

Kako čisti so ti podatki? Preverimo izstopajoče vrednosti z uporabo škatelnih diagramov. Osredotočili se bomo na številske stolpce z manj izstopajočimi vrednostmi (čeprav bi lahko odstranili izstopajoče vrednosti). Škatelni diagrami lahko pokažejo razpon podatkov in pomagajo pri izbiri, katere stolpce uporabiti. Upoštevajte, da škatelni diagrami ne prikazujejo variance, kar je pomemben element za dobro združljive podatke. Za več informacij si oglejte [to razpravo](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Škatelni diagrami](https://en.wikipedia.org/wiki/Box_plot) se uporabljajo za grafično prikazovanje porazdelitve `številskih` podatkov, zato začnimo z *izbiro* vseh številskih stolpcev skupaj s priljubljenimi glasbenimi žanri.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Opazite, kako funkcija za izbiro `where` to olajša 💁? Raziščite še druge podobne funkcije [tukaj](https://tidyselect.r-lib.org/).

Ker bomo izdelali škatlaste diagrame za vsako številsko značilnost in se želimo izogniti uporabi zank, bomo preoblikovali naše podatke v *daljšo* obliko, ki nam bo omogočila uporabo `facets` - podgrafov, ki vsak prikazujejo en podnabor podatkov.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Zdaj pa nekaj daljšega! Čas je za nekaj `ggplotov`! Kateri `geom` bomo uporabili?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Zdaj lahko vidimo, da so ti podatki nekoliko hrupni: če opazujemo vsak stolpec kot škatelni diagram, lahko vidimo odstopajoče vrednosti. Lahko bi pregledali podatkovni niz in odstranili te odstopajoče vrednosti, vendar bi to podatke precej zmanjšalo.

Zaenkrat izberimo, katere stolpce bomo uporabili za našo nalogo gručenja. Izberimo številske stolpce s podobnimi razponi. Stolpec `artist_top_genre` bi lahko kodirali kot številskega, vendar ga bomo za zdaj izpustili.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Izračun k-means gručenja v R

K-means lahko izračunamo v R z vgrajeno funkcijo `kmeans`, glejte `help("kmeans()")`. Funkcija `kmeans()` sprejme podatkovni okvir z vsemi številsko izraženimi stolpci kot svoj primarni argument.

Prvi korak pri uporabi k-means gručenja je določitev števila gručenj (k), ki bodo ustvarjene v končni rešitvi. Vemo, da obstajajo 3 glasbeni žanri, ki smo jih izluščili iz nabora podatkov, zato poskusimo s 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Kmeans objekt vsebuje več informacij, ki so dobro razložene v `help("kmeans()")`. Za zdaj se osredotočimo na nekaj ključnih točk. Vidimo, da so podatki razdeljeni v 3 skupine velikosti 65, 110, 111. Rezultat prav tako vsebuje središča skupin (povprečja) za 3 skupine glede na 5 spremenljivk.

Vektor razvrščanja predstavlja dodelitev skupine za vsako opazovanje. Uporabimo funkcijo `augment`, da dodamo dodelitev skupine v originalni nabor podatkov.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Odlično, pravkar smo razdelili naš nabor podatkov v 3 skupine. Kako dobra je torej naša razvrstitev 🤷? Poglejmo si `Silhouette score`.

### **Silhouette score**

[Silhouette analiza](https://en.wikipedia.org/wiki/Silhouette_(clustering)) se lahko uporabi za preučevanje razdalje med nastalimi grozdi. Ta ocena se giblje od -1 do 1, pri čemer vrednost blizu 1 pomeni, da je grozd gost in dobro ločen od drugih grozdov. Vrednost blizu 0 predstavlja prekrivajoče se grozde, kjer so vzorci zelo blizu odločitveni meji sosednjih grozdov. [vir](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metoda povprečnega silhouette izračuna povprečno silhouette opazovanj za različne vrednosti *k*. Visoka povprečna silhouette ocena kaže na dobro razvrstitev.

Funkcija `silhouette` v paketu za razvrščanje omogoča izračun povprečne širine silhouette.

> Silhouette se lahko izračuna z uporabo katere koli [razdalje](https://en.wikipedia.org/wiki/Distance "Distance"), kot sta [Evklidska razdalja](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") ali [Manhattanska razdalja](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), ki smo ju obravnavali v [prejšnji lekciji](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Naš rezultat je **.549**, kar pomeni, da smo nekje na sredini. To kaže, da naši podatki niso posebej primerni za tovrstno razvrščanje v skupine. Poglejmo, ali lahko to domnevo vizualno potrdimo. Paket [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) ponuja funkcije (`fviz_cluster()`), ki omogočajo vizualizacijo razvrščanja v skupine.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Prekrivanje med grozdi kaže, da naši podatki niso posebej primerni za to vrsto grozdenja, vendar nadaljujmo.

## 4. Določanje optimalnega števila grozdov

Osnovno vprašanje, ki se pogosto pojavi pri K-Means grozdenju, je naslednje - brez znanih oznak razredov, kako veste, na koliko grozdov razdeliti svoje podatke?

Eden od načinov, kako to ugotoviti, je uporaba vzorca podatkov za `ustvarjanje serije modelov grozdenja` z naraščajočim številom grozdov (npr. od 1 do 10) in ocenjevanje metrik grozdenja, kot je **Silhouette score.**

Določimo optimalno število grozdov tako, da izvedemo algoritem grozdenja za različne vrednosti *k* in ocenimo **vsoto kvadratov znotraj grozda** (WCSS). Skupna vsota kvadratov znotraj grozda (WCSS) meri kompaktnost grozdenja, pri čemer želimo, da je čim manjša, saj nižje vrednosti pomenijo, da so podatkovne točke bližje skupaj.

Raziskujmo učinek različnih izbir `k`, od 1 do 10, na to grozdenje.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Zdaj, ko imamo skupno vsoto kvadratov znotraj grozdov (tot.withinss) za vsak algoritem grozdenja s središčem *k*, uporabimo [metodo komolca](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), da najdemo optimalno število grozdov. Metoda vključuje risanje WCSS kot funkcije števila grozdov in izbiro [komolca krivulje](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") kot števila grozdov, ki jih uporabimo.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Graf prikazuje veliko zmanjšanje WCSS (torej večjo *kompaktnost*), ko se število grozdov poveča z enega na dva, in nadaljnje opazno zmanjšanje z dveh na tri grozde. Po tem je zmanjšanje manj izrazito, kar povzroči `komolec` 💪 na grafu pri približno treh grozdih. To je dober pokazatelj, da obstajata dva do trije razmeroma dobro ločeni grozdi podatkovnih točk.

Zdaj lahko nadaljujemo in izluščimo model grozdenja, kjer je `k = 3`:

> `pull()`: uporablja se za izvlečenje ene same kolone
>
> `pluck()`: uporablja se za indeksiranje podatkovnih struktur, kot so seznami


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Odlično! Poglejmo si pridobljene grozde. Vas zanima nekaj interaktivnosti z uporabo `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Morda bi pričakovali, da bo imel vsak grozd (predstavljen z različnimi barvami) različne žanre (predstavljene z različnimi oblikami).

Poglejmo natančnost modela.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Točnost tega modela ni slaba, vendar ni odlična. Morda podatki niso primerni za K-Means razvrščanje. Ti podatki so preveč neuravnoteženi, premalo povezani in med vrednostmi stolpcev je preveč variance, da bi jih lahko dobro razvrstili. Pravzaprav so skupine, ki se oblikujejo, verjetno močno vplivane ali izkrivljene zaradi treh kategorij žanrov, ki smo jih opredelili zgoraj.

Kljub temu je bil to precej poučen proces!

V dokumentaciji Scikit-learn lahko vidite, da ima model, kot je ta, pri katerem skupine niso dobro opredeljene, težavo z 'varianco':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika iz Scikit-learn</figcaption>



## **Varianca**

Varianca je opredeljena kot "povprečje kvadratov razlik od povprečja" [vir](https://www.mathsisfun.com/data/standard-deviation.html). V kontekstu te težave z razvrščanjem se nanaša na podatke, pri katerih se vrednosti našega nabora podatkov preveč oddaljujejo od povprečja.

✅ To je odličen trenutek, da razmislite o vseh načinih, kako bi lahko odpravili to težavo. Bi lahko podatke še malo prilagodili? Uporabili različne stolpce? Uporabili drugačen algoritem? Namig: Poskusite [normalizirati podatke](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) in preizkusiti druge stolpce.

> Poskusite '[kalkulator variance](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', da bolje razumete koncept.

------------------------------------------------------------------------

## **🚀Izziv**

Preživite nekaj časa s tem zvezkom in prilagodite parametre. Ali lahko izboljšate natančnost modela z dodatnim čiščenjem podatkov (na primer odstranjevanjem odstopanj)? Lahko uporabite uteži, da nekaterim vzorcem podatkov dodelite večjo težo. Kaj še lahko storite, da ustvarite boljše skupine?

Namig: Poskusite normalizirati podatke. V zvezku je komentirana koda, ki dodaja standardno normalizacijo, da se stolpci podatkov bolj približajo drug drugemu glede na obseg. Ugotovili boste, da se medtem ko se silhuetni rezultat zniža, 'pregib' na grafu komolca zgladi. To je zato, ker nenormalizirani podatki omogočajo, da podatki z manjšo varianco nosijo večjo težo. Preberite več o tej težavi [tukaj](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kvizi po predavanju**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Pregled in samostojno učenje**

-   Oglejte si simulator K-Means [kot je ta](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). S tem orodjem lahko vizualizirate vzorčne podatkovne točke in določite njihove centroidne točke. Lahko urejate naključnost podatkov, število skupin in število centroidov. Ali vam to pomaga pridobiti idejo, kako se podatki lahko razvrstijo?

-   Prav tako si oglejte [ta priročnik o K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) iz Stanforda.

Želite preizkusiti svoje novo pridobljene veščine razvrščanja na naborih podatkov, ki so primerni za K-Means razvrščanje? Oglejte si:

-   [Usposabljanje in ocenjevanje modelov razvrščanja](https://rpubs.com/eR_ic/clustering) z uporabo Tidymodels in podobnih orodij

-   [Analiza skupin K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-Means razvrščanje z načeli urejenih podatkov](https://www.tidymodels.org/learn/statistics/k-means/)

## **Naloga**

[Preizkusite različne metode razvrščanja](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## HVALA:

[Jen Looper](https://www.twitter.com/jenlooper) za ustvarjanje izvirne Python različice tega modula ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) za ustvarjanje čudovitih ilustracij, ki naredijo R bolj prijazen in privlačen. Več ilustracij najdete v njeni [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Veselo učenje,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Umetniško delo @allison_horst</figcaption>



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
